In [31]:
from geostructures import *
from geostructures.collections import FeatureCollection, Track
from datetime import datetime, timedelta
from geostructures.geohash import H3Hasher

import pandas as pd
hasher = H3Hasher(resolution=10)

# To create a FeatureCollection, simply pass it a list of geoshapes
fcol = FeatureCollection(
    [
        GeoCircle(Coordinate(-0.118092, 51.509865), 500, dt=datetime(2020, 1, 1, 9, 5)),
        GeoEllipse(Coordinate(-0.118072, 51.502365), 2000, 400, 80, dt=datetime(2020, 1, 1, 9, 5)),
        GeoCircle(Coordinate(-0.141092, 51.529865), 500, dt=datetime(2020, 1, 1, 9, 5)),
        GeoRing(Coordinate(-0.116092, 51.519865), inner_radius=800, outer_radius=1000, dt=datetime(2020, 1, 1, 10, 5)),
        GeoRing(Coordinate(-0.101092, 51.514865), inner_radius=300, outer_radius=500, angle_min=60, angle_max=190, dt=datetime(2020, 1, 2, 11, 5)),
    ]
)



In [3]:
track = Track(fcol.geoshapes)

In [4]:
track.end

datetime.datetime(2020, 1, 2, 11, 5, tzinfo=datetime.timezone.utc)

In [ ]:
from datetime import datetime, timedelta

# list of datetime objects
datetime_list = [datetime(2022, 1, 1), datetime(2022, 1, 2), datetime(2022, 1, 3)]
print("Original list: ", datetime_list)

# change the last value
datetime_list[-1] = datetime_list[-1] + timedelta(seconds=1)
print("After changing the last value: ", datetime_list)


In [52]:
def get_timestamp_intervals(track: Track, hour_interval: int):
    """
    gets the timestamps for a Track partitioned by a specified length
    of time in hours  
    
    Args:
        track: the target Track

        hour_interval= the length in hours of the desired interval

    Returns:
        A list of timestamps starting from an interval from the start
    """
    start_time = track.start
    end_time = track.end
    timestamps = []

    # get timestamp intervals
    date_x = start_time
    while date_x < end_time:
        date_x += timedelta(hours = hour_interval)
        timestamps.append(date_x)
    
    # change the last value to be inclusive 
    timestamps[-1] = timestamps[-1] + timedelta(seconds=1)
    
    
    return timestamps

In [57]:
def time_slice_track(track: Track, timestamps):
    """
    Slices a Track into several tracks that are partitioned by a list
    of time stamps  
    
    Args:
        track: the target Track

        timestamps= a list of timestamps

    Returns:
        A list of tracks
    """
    start_time = track.start
    sliced_tracks = []

    
    for timestamp in timestamps:
        sliced_track = track[start_time: timestamp]
        start_time = timestamp
        sliced_tracks.append(sliced_track)
    
    return sliced_tracks

In [77]:
def hash_tracks_into_df(track_list , timestamps, hash_func):
    """
    Converts a list of tracks into a pandas dataframe using
    a specified hashing function with intervals reflected
    in a corresponding timestamp list 
    
    Args:
        track_list: a list of tracks broken down by equal intervals

        timestamps: a list of corresponding timestamps

        hash_func: the hashing function

    Returns:
        A pandas dataframe
    """
    interval_start = track_list[0].start
    row_list = []
    for track,timestamp  in zip(track_list, timestamps):
        hashmap = hash_func(track)
        start_string= interval_start.strftime("%Y-%m-%d %H:%M:%S")
        end_string= timestamp.strftime("%Y-%m-%d %H:%M:%S")
        interval = start_string + ", " + end_string
        new_row = pd.Series(data=hashmap, name=interval)
        row_list.append(new_row)
        interval_start = timestamp
    
    df = pd.DataFrame(row_list)
    
    return df

In [53]:
timestamps = get_timestamp_intervals(track, 1)

In [61]:
test=time_slice_track(track, timestamps)

In [78]:
hash_track_into_df(test, timestamps, hasher.hash_collection)

,8a194ad3219ffff,8a194ad32797fff,8a194ad3254ffff,8a194ad3240ffff,8a194ad3271ffff,8a194ad32457fff,8a194ad32727fff,8a194ad3272ffff,8a194ad32607fff,8a194ad3255ffff,...,8a194ad32b27fff,8a194ad304b7fff,8a194ad32b17fff,8a194ad3049ffff,8a194ad32b1ffff,8a194ad32b07fff,8a194ad32b8ffff,8a194ad3294ffff,8a194ad32b9ffff,8a194ad30497fff
"2020-01-01 09:05:00, 2020-01-01 10:05:00",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 10:05:00, 2020-01-01 11:05:00",NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 11:05:00, 2020-01-01 12:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 12:05:00, 2020-01-01 13:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 13:05:00, 2020-01-01 14:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 14:05:00, 2020-01-01 15:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 15:05:00, 2020-01-01 16:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 16:05:00, 2020-01-01 17:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 17:05:00, 2020-01-01 18:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2020-01-01 18:05:00, 2020-01-01 19:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
test

[<Track with 3 shapes from 2020-01-01T09:05:00+00:00 - 2020-01-01T09:05:00+00:00>,
 <Track with 1 shapes from 2020-01-01T10:05:00+00:00 - 2020-01-01T10:05:00+00:00>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Empty Track>,
 <Track with 1 shapes from 2020-01-02T11:05:00+00:00 - 2020-01-02T11:05:00+00:00>]

In [ ]:
from datetime import datetime, timedelta
date_a = datetime(2016, 8, 9, 1, 24, 30, 993352)
date_b = datetime(2016, 8, 9, 7, 24, 30, 993352)

delta = timedelta(hours=1)
five_min_timestamps = []

date_x = date_a
while date_x < date_b:
    date_x += delta
    five_min_timestamps.append(date_x)

print(five_min_timestamps)

[datetime.datetime(2016, 8, 9, 2, 24, 30, 993352), datetime.datetime(2016, 8, 9, 3, 24, 30, 993352), datetime.datetime(2016, 8, 9, 4, 24, 30, 993352), datetime.datetime(2016, 8, 9, 5, 24, 30, 993352), datetime.datetime(2016, 8, 9, 6, 24, 30, 993352), datetime.datetime(2016, 8, 9, 7, 24, 30, 993352)]
